In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
commits = pd.read_csv('commits.csv', index_col=0, parse_dates=['date'])
df = commits.sort_values(by='date').reset_index().drop(columns=['index'])

In [ ]:
df['weight'] = df['ins'] + df['del']

In [ ]:
# sum lines by commit (drop repo/file information)
df1 = df.drop(columns=['ins', 'del', 'dir', 'path']).groupby(['date', 'email']).sum()
# sum previous weight values, by email, for plotting
df1['contrib'] = df1.groupby('email')['weight'].transform(pd.Series.cumsum)
# convert data, email indexes to columns
df2 = df1.reset_index()

In [ ]:
fig = px.line(df2, x='date', y='contrib', color='email', markers=True)
fig.show()